**Runtime → Change runtime type → make sure GPU is selected.**

**1) Install libraries**

In [1]:
!pip install -q torch transformers accelerate bitsandbytes sqlparse pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.7 MB/s eta 0:00:00


**2) Verify GPU + memory (helps us pick 8-bit vs fp16)**

In [2]:
import torch, math
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    m = torch.cuda.get_device_properties(0).total_memory
    print("GPU total memory (GB):", round(m/1e9, 2))


CUDA available: True
GPU total memory (GB): 15.83


**3) Load tokenizer + model (memory-aware)**

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "defog/sqlcoder-7b-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

use_8bit = True  # default safe; switch to fp16 if plenty of VRAM
if torch.cuda.is_available():
    total_mem = torch.cuda.get_device_properties(0).total_memory
    # if you have ~20GB+ VRAM, fp16 is usually fine
    if total_mem >= 20e9:
        use_8bit = False

if use_8bit:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        device_map="auto",
        load_in_8bit=True,
        use_cache=True,
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )

# make sure we have a pad token (some LLMs don’t define one)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

**4) Define your database schema (cleaned)**

In [4]:
SCHEMA = """
CREATE TABLE products(
  product_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  price DECIMAL(10,2),
  quantity INTEGER
);

CREATE TABLE salespeople(
  salesperson_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  region VARCHAR(50)
);

CREATE TABLE sales(
  sale_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  customer_id INTEGER,
  salesperson_id INTEGER,
  sale_date DATE,
  quantity INTEGER
);

CREATE TABLE product_supplier(
  supplier_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  supply_price DECIMAL(10,2)
);

-- Joins:
-- sales.product_id -> products.product_id
-- sales.salesperson_id -> salespeople.salesperson_id
-- product_supplier.product_id -> products.product_id
"""


**5) Build a solid prompt template**

In [5]:
PROMPT = """### Task
Generate a SQL query to answer the user's question using ONLY the provided database schema.

### Rules
- If the question cannot be answered with the schema, return exactly: I do not know
- revenue = price * quantity
- cost = supply_price * quantity
- Return only the SQL after the [SQL] tag.

### Database Schema
{schema}

### Question
{question}

### Answer
[SQL]
"""


**6) Helper to extract/format SQL**

In [6]:
import re, sqlparse

def extract_sql(text: str) -> str:
    """
    Pulls SQL after the [SQL] tag. If not found, tries to find first SELECT.
    Returns 'I do not know' if nothing usable is found.
    """
    after = text.split("[SQL]", 1)[-1] if "[SQL]" in text else text
    # strip special tokens
    after = after.replace("</s>", " ").replace("<s>", " ").strip()

    # try to find a SQL-y start
    m = re.search(r"(SELECT|WITH)\b.*", after, flags=re.IGNORECASE | re.DOTALL)
    if not m:
        # sometimes models add backticks or code fences
        m = re.search(r"```(?:sql)?(.*)```", after, flags=re.IGNORECASE | re.DOTALL)
        if m:
            candidate = m.group(1).strip()
        else:
            return "I do not know"
    else:
        candidate = m.group(0).strip()

    # basic hard stop at first semicolon block (optional)
    # keep multi-statement if needed; here we keep as-is
    formatted = sqlparse.format(candidate, reindent=True)
    return formatted if formatted else "I do not know"


**7) The generate_query() function**

In [7]:
def generate_query(question: str, max_new_tokens: int = 300) -> str:
    prompt = PROMPT.format(schema=SCHEMA, question=question)

    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,            # deterministic for consistency
        num_beams=1,                # beam=1 is fine for structured outputs
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
    sql = extract_sql(text)

    # optional: guardrail — if the model didn’t produce SELECT/WITH, force "I do not know"
    if not re.search(r"^(SELECT|WITH)\b", sql, flags=re.IGNORECASE):
        return "I do not know"
    return sql


**8) Test with a single question**

In [8]:
question = "What was the highest quantity sold last month?"
sql = generate_query(question)
print(sql)


SELECT MAX(s.quantity) AS max_quantity
FROM sales s
WHERE s.sale_date >= (CURRENT_DATE - INTERVAL '1 month');
